In [1]:
import numpy as np
import os
import pandas as pd
import re

from Bio import SeqIO
from collections import Counter, OrderedDict
from itertools import permutations

In [157]:
# parent directories

COORDS_DIR = "./coords"			# directory containing coordinate files
EXTRACTED_DIR = "./extracted"	# directory containing extracted .vcf files (Section 1)


WINDOWS_DIR = "./windows"		# directory to write out windows (Section 2)
MATCHES_DIR = "./matches"		# directory to write out matches (Section 3)
COUNTS_DIR = "./counts"			# directory to write out counts (Section 5)
METRICS_DIR = "./metrics"		# directory to write out metrics (Sections 7, 8)

check_dir = WINDOWS_DIR, MATCHES_DIR, COUNTS_DIR, METRICS_DIR
for directory in check_dir:
	if not os.path.exists(directory):
		os.mkdir(directory)

In [158]:
# subdirectories

POS_COUNTS_SUBDIR = f"{COUNTS_DIR}/positional"			# positional counts subdirectorty (Section 5)
POSITIONAL_METRICS_SUBDIR = f"{METRICS_DIR}/positional"	# positional metrics subdirectorty (Section 7)
GENE_LEVEL_METRICS_SUBDIR = f"{METRICS_DIR}/gene_level"	# gene level metrics subdirectorty (Section 8)

check_dir = POS_COUNTS_SUBDIR, POSITIONAL_METRICS_SUBDIR, GENE_LEVEL_METRICS_SUBDIR
for directory in check_dir:
	if not os.path.exists(directory):
		os.mkdir(directory)

In [ ]:
# paths to coordinate files
TSS_COORDS_PATH = f"{COORDS_DIR}/Fantom5_CAGE_hg38_gencode.v40_TSS_EIB_Updated_200.txt"
INTERGEN_GCMATCH_COORDS_PATH = f"{COORDS_DIR}/hg38_intergenic_more_1.txt"
INTERGEN_NO_GCMATCH_COORDS_PATH = f"{COORDS_DIR}/hg38_intergenic_notGCmatched.txt"

# path to genome FASTA file
HG38_PATH = "./Homo_sapiens.GRCh38.dna.toplevel.fa"

In [3]:
BASES_SET = {'A','C','G','T'}   # set of valid characters representing the four bases
CPG_OFFSET = 1                  # window offset to account for CpGs at the edges of windows
CPG_STR = 'CG'                  # string representation of CpG
INTERGEN_GCMATCH_WINLEN = 100	# length of intergenic GC matched window
RANDOM_STATE  = 1337            # random state/seed for random number generation
SLIDING_WIN_LEN = 100			# length of sliding window used for generating metrics with a sliding window
WINDOW_LENGTH = 5_000           # length of window

# dictionary to map chromosome identifier format; "chrom#" -> # (i.e. "chrom9" -> 9)
CHROM_DICT = { f"chr{i}":i
				for i
				in range(1,24) }
CHROM_DICT.update({
	'chrX':'X',
	'chrY':'Y',
	'chrM':'M',
})

# set of possible mutations (i.e. A>G, C>T, etc.)
DNM_TYPES = { '>'.join(ref_alt)
				for ref_alt
				in (list(permutations(BASES_SET, 2))) }

In [ ]:
# Global writing mode option. This sets the behavior when writing results out to disk.
# There are two options:
# 'w' = create new file if doesn't exist, or overwrite file exist
# 'x' = create new file if doesn't exist, but does NOT overwrite file if exists

WRITING_MODE = 'x'

___________

# **1.** <u>Iceland dataset</u>

## **1.1** Download compressed files

This dataset is at https://www.ebi.ac.uk/ena/browser/view/PRJEB21300. The *Download All* link downloads a shell script that downloads the *.vcf.gz* (compressed DNM information) and *.vcf.gz.tbi* (supporting information on the whole dataset; not useful for our purposes) files for each proband. To only download the *.vcf.gz* files, the cell below strips out the command to download the *.vcf.gz.tbi* files.

*Note: the filename (i.e. "ena-file-download-20230531-0107.sh") is timestamped, thus its name is dependent on the time it was downloaded. Change this to the name of the file downloaded by you.*

*Note: this shell script is easy to run on Linux but must be given executable permissions befor it can be run.*

In [ ]:
shellscript_path = "./ena-file-download-20230531-0107.sh"    # CHANGE THIS LINE TO POINT TO THE FILE YOU DOWNLOADED

with open(f"{shellscript_path}", 'r') as f:
	lines_read = [ line
					for line
					in f.readlines()
					if not line.endswith(".tbi\n") ]

with open(f"{shellscript_path.strip('.sh')}_stripped.sh", WRITING_MODE) as f:
	f.writelines(lines_read)

After running the script and downloading the compressed files, the files were manually extracted to `EXTRACTED_DIR`. The following section cleans up the files and converts them to comma separated files (*.csv*).

## **1.2** Read *.vcf* files

This sub-section reads the files into a single list, creates a Pandas DataFrame from the list, and writes the dataset to a file called "iceland_dataset.csv" in a comma-separated format.

Please note, this does not preserve proband information. The only information that is preserved is individual DNM information. That is, only the following DNM data is kept from the original *.vcf* files:

1. chromosome ("CHROM")
2. position ("POS")
3. reference base ("REF")
4. altered base ("ALT")

The following is ignored from each *.vcf* file:
- comment section, i.e. lines starting with '##'
- column headers, i.e. line starting with '#'
- the following columns:
	- "ID"
	- "QUALITY"
	- "FILTER"
	- "INFO"
	- "FORMAT"
	- "Proband-#" (where # is the proband number)

Also note: DNMs with reference or altered bases greater than 1 base in length are discarded. Only single base mutations are used for the rest of this analysis.

In [53]:
# parse folder containing extracted .vcf files

dir_list = [ f
			  for f
			  in sorted(os.listdir(EXTRACTED_DIR))
			  if f.endswith(".vcf") ]

In [142]:
# extract DNM information from each file: chromosome, position, reference base at position, base found at position in proband
# for example:
# > print(aggregation)
#   [['CHROM', 'POS', 'REF', 'ALT'],
#   [1, '2360454', 'C', 'A'],
#               .
#               .
#               .
#   [22, '26576171', 'C', 'T']]

aggregation = list()
aggregation.append([    # these will be column headers for the dataset
	'CHROM',
	'POS',
	'REF',
	'ALT',
])

info_idx = 0,1,3,4  # columns for CHROM, POS, REF, ALT

for i, fname in enumerate(dir_list, start=1):
	with open(f"{EXTRACTED_DIR}/{fname}", 'r') as f:
		# extract DNM information from each line of the file
		lines_read = [ [    str.split(line,'\t')[index] if index else CHROM_DICT[str.split(line,'\t')[index]] # convert string chromosome ID to integer ID (i.e. 'chrom1' to 1)
							for index
							in info_idx     ]
						for line
						in f.readlines()
						if not line[0].startswith('#') ]    # skip lines starting with '#', as they are comments

		aggregation.extend(lines_read)
		
	print(f"{i*100/len(dir_list):.0f}% complete", end='\r')

In [144]:
# load aggregation into Pandas dataset

iceland_columns = aggregation[0] # use first item in aggregation as column headers
iceland_dataset = pd.DataFrame(
	aggregation[1:],
	columns=iceland_columns,
)

# discard mutations greater than 1 base in length
iceland_dataset = iceland_dataset[
	(iceland_dataset.REF.apply(len) == 1) &
	(iceland_dataset.ALT.apply(len) == 1)
]
iceland_dataset.CHROM = iceland_dataset.CHROM.astype(str)   # ensure POS is integer
iceland_dataset.POS = iceland_dataset.POS.astype(int)   # ensure POS is integer

iceland_dataset

,CHROM,POS,REF,ALT
0,1,619713,C,T
1,1,27343574,G,A
2,1,41444960,A,G
3,1,47323686,C,T
4,1,48889441,A,T
...,...,...,...,...
108649,8,140426803,C,G
108650,9,14254838,A,T
108652,9,102594206,C,T
108653,9,113945174,G,A


In [7]:
# write dataset out to file
with open("./iceland_dataset.csv", WRITING_MODE) as fout:
	iceland_dataset.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

_________

# **2.** <u>Generate windows</u>

This section generates ±2501b windows around the TSS/intergenic window center for each coordinate. The desired window length is ±2.5kb, for a total window size of 5kb, however, 1 extra base is included on either edge of the window. This allows for counting of CpG mutations at the edges.

## **2.1** Load hg38 genome

This sub-section loads the reference genome into a Biopython dictionary object. The reference genome used in this analysis is the *hg38 genome, release 109*. The FASTA file for this genome can be accessed through the following link:

https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz

*Please note, `SeqIO.index()` function is slow — this function can take upward of 10 minutes for the FASTA file used in this analysis. Also note, `SeqIO.index()` is very slow to reference items within it as it loads the genome dynamically into memory on demand from disk, which is a very slow process. Use `SeqIO.to_dict()` if you have large RAM (>70GB) for significantly faster indexing, as `SeqIO.to_dict()` loads the entire genome into memory. Both are given in the cell below, but use ONLY one and NOT both.*

In [12]:
# use this if you have smaller RAM capacity
hg38 = SeqIO.index(
	HG38_PATH,
	'fasta',
)

# # ONLY use this if you have >70GB of RAM
# hg38 = SeqIO.to_dict(
#     SeqIO.parse(
#         HG38_PATH,
#         'fasta',
#     ),
# )

## **2.2** Read coordinates

This sub-section reads the coordinates for each coordinate set into a Pandas DataFrame.

In [14]:
tss_coords_columns = [  # column headers
	"CHROM",
	"TSS",
	"FIRST_EIB",
	"STRAND",
	"ENSGID",
	"ENSTID",
]

tss_coords = pd.read_csv(
	TSS_COORDS_PATH,
	delimiter='\t',
	skiprows=[0],
	names=tss_coords_columns,
)
tss_coords.replace({'CHROM':CHROM_DICT}, inplace=True)  # chromosome string formatting ("chrom#" -> # ; i.e. 'chrome1' -> 1)
tss_coords = tss_coords[tss_coords.CHROM != 'M']        # drop mitochondrial DNA DNMs 
tss_coords.TSS = tss_coords.TSS.astype(int)             # ensure TSS is integer
tss_coords.FIRST_EIB = tss_coords.FIRST_EIB.astype(int) # ensure FIRST_EIB is integer
tss_coords = tss_coords.drop_duplicates()

tss_coords

,CHROM,TSS,FIRST_EIB,STRAND,ENSGID,ENSTID
0,1,923923,924948,+,ENSG00000187634,ENST00000616016
1,1,959256,959215,-,ENSG00000188976,ENST00000327044
2,1,960584,960800,+,ENSG00000187961,ENST00000338591
3,1,966502,966614,+,ENSG00000187583,ENST00000379407
4,1,1000097,999866,-,ENSG00000188290,ENST00000304952
...,...,...,...,...,...,...
17101,Y,14056227,14056391,+,ENSG00000129862,ENST00000250823
17102,Y,14524529,14524708,+,ENSG00000165246,ENST00000481089
17103,Y,19744726,19744671,-,ENSG00000012817,ENST00000317961
17104,Y,20575776,20575887,+,ENSG00000198692,ENST00000361365


In [15]:
intergen_coords_columns = [ # column headers
	"CHROM",
	"START",
	"END",
]

intergen_GCmatch_coords = pd.read_csv(
	INTERGEN_GCMATCH_COORDS_PATH,
	delimiter='\t',
	skiprows=[0],
	names=intergen_coords_columns,
)
intergen_GCmatch_coords.replace({'CHROM':CHROM_DICT}, inplace=True)         # chromosome string formatting ("chrom#" -> # ; i.e. 'chrome1' -> 1)
intergen_GCmatch_coords.CHROM = intergen_GCmatch_coords.CHROM.astype(str)   # ensure CHROM is string
intergen_GCmatch_coords.START = intergen_GCmatch_coords.START.astype(int)   # ensure START is integer
intergen_GCmatch_coords.END = intergen_GCmatch_coords.END.astype(int)       # ensure END is integer
intergen_GCmatch_coords = intergen_GCmatch_coords.drop_duplicates()

intergen_GCmatch_coords

,CHROM,START,END
0,1,114327737,114327837
1,1,114328137,114328237
2,1,114328237,114328337
3,1,47396304,47396404
4,1,166420304,166420404
...,...,...,...
17107,21,8439401,8439501
17108,22,44331786,44331886
17109,22,46071211,46071311
17110,22,16601785,16601885


In [16]:
intergen_nonGCmatch_coords = pd.read_csv(
	INTERGEN_NO_GCMATCH_COORDS_PATH,
	delimiter='\t',
	skiprows=[0],
	names=intergen_coords_columns,
)
intergen_nonGCmatch_coords.replace({'CHROM':CHROM_DICT}, inplace=True)          # chromosome string formatting ("chrom#" -> # ; i.e. 'chrome1' -> 1)
intergen_nonGCmatch_coords.CHROM = intergen_nonGCmatch_coords.CHROM.astype(str) # ensure CHROM is string
intergen_nonGCmatch_coords.START = intergen_nonGCmatch_coords.START.astype(int) # ensure START is integer
intergen_nonGCmatch_coords.END = intergen_nonGCmatch_coords.END.astype(int)     # ensure END is integer
intergen_nonGCmatch_coords = intergen_nonGCmatch_coords.drop_duplicates()

intergen_nonGCmatch_coords

,CHROM,START,END
0,1,128802631,128807631
1,1,139115639,139120639
2,1,114326537,114331537
3,1,47396104,47401104
4,1,166419304,166424304
...,...,...,...
177995,22,29301107,29306107
177996,22,6327835,6332835
177997,22,470324,475324
177998,22,11498452,11503452


## **2.3** Generate windows

This sub-section generates a string representation of the ±2501 base window centered around each coordinate.

In [168]:
# drop coordinates with windows containing bases other than 'A', 'C', 'G', 'T'

def drop_invalid_windows(windows:pd.DataFrame, valid_bases:set[str], window_col:int=-1, print_dropped:bool=True) -> None:
	'''
	Drop coordinates inplace with windows containing bases other than those specified in `valid_bases`.

	## Parameters
	windows: DataFrame
		DataFrame containing windows
	valid_bases: set
		Set of strings representing valid bases
	window_col: int, default = -1
		Column index in `windows` containing the window string
	print_dropped: bool, default = True
		If True, indices of dropped windows, along with a set containing
		the invalid bases found in the window, are printed
	'''

	any_dropped = False

	for i, coord in enumerate(windows.values):
		window = coord[window_col]

		if not set(window) <= valid_bases:
			windows.drop(i, inplace=True)
			
			if print_dropped:
				if not any_dropped:
					print("Dropping", end=' ')
					any_dropped = True
				print(f"{i} (contains invalid base {set(window) - valid_bases})", end=', ')

	if not any_dropped:
		print("No windows dropped")

### **2.3.1** TSS windows

In this sub-sub-section, the TSS is used as the center.

This method packs the data together in a list (`tss_windows`) of lists. Each sublist contains the following information for each coordinate: the chromosome, tss, first eib, stradedness, ensGID, ensTID, and 5kb window.

Different offsets for positive and negative strands are accounted for. Positive strands are indexed as [-2501, 2500) around the TSS. Negative strands are indexed as [-2500, 2501) around the TSS, and are reversed. This keeps the TSS at index 0 when interpreting the range as [-2501,2500]. The left half of the window is the intergenic space immediately before the gene, and the right half is the gene itself.

In [169]:
tss_windows = list()
for chrom, tss, first_eib, strand, ensgid, enstid in tss_coords.values:
	assert strand in '+-'

	chrom_value = hg38[str(chrom)]
	if strand == '+':
		start_idx = tss - (WINDOW_LENGTH//2 + 1) - CPG_OFFSET
		stop_idx = tss + (WINDOW_LENGTH//2 - 1) + CPG_OFFSET
		direction = 1

	elif strand == '-':
		start_idx = tss - WINDOW_LENGTH//2 - CPG_OFFSET
		stop_idx = tss + WINDOW_LENGTH//2 + CPG_OFFSET
		direction = -1


	tss_windows.append(
		[
			chrom,
			tss,
			first_eib,
			strand,
			ensgid,
			enstid,
			str(chrom_value[start_idx : stop_idx].seq)[::direction],   # string representation of ±2501 base window
		]
	)


columns = [	# column headers
	'CHROM',
	'TSS',
	'FIRST_EIB',
	'STRAND',
	'ENSGID',
	'ENSTID',
	'WINDOW',
]
tss_windows = pd.DataFrame(
	tss_windows,
	columns=columns,
)

# explicit casting of column types to ensure consistent results
tss_windows.CHROM = tss_windows.CHROM.astype(str)
tss_windows.TSS = tss_windows.TSS.astype(int)
tss_windows.FIRST_EIB = tss_windows.FIRST_EIB.astype(int)

tss_windows

,CHROM,TSS,FIRST_EIB,STRAND,ENSGID,ENSTID,WINDOW
0,1,923923,924948,+,ENSG00000187634,ENST00000616016,GGGAGAGTGGGGCCCAGGTGTCCCAGACCAAGACAGGTGTGTGAGT...
1,1,959256,959215,-,ENSG00000188976,ENST00000327044,GGGAGTGGACGTGTAACGGGAGCGGGTGGTGTTAGAGTCGGCACAT...
2,1,960584,960800,+,ENSG00000187961,ENST00000338591,TCCTGAGGCGGAATCTCACTCTGTCGCCCAGGCTGGAGTGCAGTGG...
3,1,966502,966614,+,ENSG00000187583,ENST00000379407,CACGCTTGGTGGGTGATGGGGCCTGCCTGGGGGGCATCCCCACCTT...
4,1,1000097,999866,-,ENSG00000188290,ENST00000304952,GGGGTAGAGATGATTTTTATGTTTTGTGATCGGCCCGCACCACCAT...
...,...,...,...,...,...,...,...
17101,Y,14056227,14056391,+,ENSG00000129862,ENST00000250823,TAAAAATAAAATCACATTCTTCTCACCACGTCCTGCTGAGTTATTG...
17102,Y,14524529,14524708,+,ENSG00000165246,ENST00000481089,GAGTATTTAGAAACAACTGCCTCTTGGTAAGCACACCTCCTATATT...
17103,Y,19744726,19744671,-,ENSG00000012817,ENST00000317961,ACAGAGTCAGGGTCGAACGAACAGACAACTTTGGGGTCGAACAAAG...
17104,Y,20575776,20575887,+,ENSG00000198692,ENST00000361365,CTGTCTTCTTCTAGGCCCTTCAAACTGTTCCAACCTCCCCCCATTA...


In [170]:
# drop coordinates with windows containing bases other than 'A', 'C', 'G', 'T'

drop_invalid_windows(
	windows=tss_windows,
	valid_bases=BASES_SET,
)

Dropping 8440 (contains invalid base {'N'}), 8441 (contains invalid base {'N'}), 

In [21]:
# write out results

with open(f"{WINDOWS_DIR}/hg38_tss.csv", WRITING_MODE) as fout:
	tss_windows.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

### **2.3.2** Intergenic windows

In this sub-sub-section, the intergenic coordinate is used as the center, for both the GC matched and non-GC matched coordinates. The same method used in sub-sub-section 2.3.1 for the TSS coordinates is applied here.

In [178]:
def get_intergen_windows(coords: pd.DataFrame, genome: dict, win_len: int, offset: int=0) -> list:
	'''
	Generate windows at center of coordinates specified in `coords`.

	## Parameters
	coords: Pandas.DataFrame
		Pandas.DataFrame containing coordinates.
	genome: dict
		Dictionary returned by Biopython's SeqIO interface. Contains the genome to 
		be used to generate the windows.
	win_len: int
		Length of window.
	offset: int, default = 0
		Window offset.

	## Returns
	windows: list
		List with each item containing the following information for each coordinate: 
		
			- chromosome
			- coordinate for center of window
			- window
	'''

	windows = list()
	for chrom, start, end in coords.values:
		chrom_value = genome[str(chrom)]

		start_idx = start + (end - start - win_len)//2 - offset
		stop_idx = start + (end - start + win_len)//2 + offset

		windows.append(
			[
				chrom,
				start + (end - start)//2,
				str(chrom_value[start_idx : stop_idx].seq),   # string representation of ±2501 base window
			]
		)

	return windows

In [ ]:
intergen_windows_columns = [	# column headers
	'CHROM',
	'CENTER',
	'WINDOW',
]

#### *GC matched*

In [179]:
intergen_GCmatch_windows = get_intergen_windows(
    coords=intergen_GCmatch_coords,
    genome=hg38,
    win_len=WINDOW_LENGTH,
	offset=CPG_OFFSET,
)
intergen_GCmatch_windows = pd.DataFrame(
	intergen_GCmatch_windows,
	columns=intergen_windows_columns,
)

# explicit casting of column types to ensure consistent results
intergen_GCmatch_windows.CHROM = intergen_GCmatch_windows.CHROM.astype(str)
intergen_GCmatch_windows.CENTER = intergen_GCmatch_windows.CENTER.astype(int)

intergen_GCmatch_windows

,CHROM,CENTER,WINDOW
0,1,114327787,GATAGCAGCAGTTCTTATTTTTTGAAGCCAATATTACCTTGATATA...
1,1,114328187,ACATCATACTCAGTAGTGAAGGACTAAATGCTTTCTCAAATACCTG...
2,1,114328287,GAACAATAAAGAAAGAAAAATAATTAACAGGCTTATAGATTGGAAA...
3,1,47396354,TTGCTTTTTGCACAATGGTAACACATATGGACTCCCTCTTTTTTTT...
4,1,166420354,GTGTTGAAAGATAGCACTCATCCTGATGAGTTAAACAGTTAAAGGT...
...,...,...,...
16611,21,8439451,TACAGTGAAACTGCGAATGGCTCATTAAATCAGTTATGGTTCCTTT...
16612,22,44331836,TCACAGGTCTGAGTGAGCAGCACTGGCCCTGCCCCCTACCCCTCTC...
16613,22,46071261,ATTGCCCTTGAGAACAGTTTTATTGAGAATAGTAACCATATATTTA...
16614,22,16601835,AATTTAAAATCCCATATTTAACATGTATACTTATTAGAAAGTACAC...


In [ ]:
# drop coordinates with windows containing bases other than 'A', 'C', 'G', 'T'

drop_invalid_windows(
	windows=intergen_GCmatch_windows,
	valid_bases=BASES_SET,
)

Dropping 5084 (contains invalid base {'N'}), 5085 (contains invalid base {'N'}), 5951 (contains invalid base {'N'}), 8565 (contains invalid base {'N'}), 8566 (contains invalid base {'N'}), 8567 (contains invalid base {'N'}), 8568 (contains invalid base {'N'}), 8569 (contains invalid base {'N'}), 8570 (contains invalid base {'N'}), 9881 (contains invalid base {'N'}), 9882 (contains invalid base {'N'}), 9883 (contains invalid base {'N'}), 10341 (contains invalid base {'N'}), 10810 (contains invalid base {'N'}), 10811 (contains invalid base {'N'}), 10878 (contains invalid base {'N'}), 10879 (contains invalid base {'N'}), 12246 (contains invalid base {'N'}), 13132 (contains invalid base {'N'}), 13133 (contains invalid base {'N'}), 13134 (contains invalid base {'N'}), 13135 (contains invalid base {'N'}), 13136 (contains invalid base {'N'}), 13137 (contains invalid base {'N'}), 13548 (contains invalid base {'N'}), 14915 (contains invalid base {'N'}), 15010 (contains invalid base {'N'}), 1501

In [28]:
# write out results

with open(f"{WINDOWS_DIR}/hg38_intergen_GCmatch.csv", WRITING_MODE) as fout:
	intergen_GCmatch_windows.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

#### *non-GC matched*

In [181]:
intergen_nonGCmatch_windows = get_intergen_windows(
    coords=intergen_nonGCmatch_coords,
    genome=hg38,
    win_len=WINDOW_LENGTH,
	offset=CPG_OFFSET,
)
intergen_nonGCmatch_windows = pd.DataFrame(
	intergen_nonGCmatch_windows,
	columns=intergen_windows_columns,
)
intergen_nonGCmatch_windows = intergen_nonGCmatch_windows[~intergen_nonGCmatch_windows.WINDOW.str.contains('N')]	# explicitly drop rows with windows containing 'N' for ease of filtering for other illegal bases

# explicit casting of column types to ensure consistent results
intergen_nonGCmatch_windows.CHROM = intergen_nonGCmatch_windows.CHROM.astype(str)
intergen_nonGCmatch_windows.CENTER = intergen_nonGCmatch_windows.CENTER.astype(int)

intergen_nonGCmatch_windows

,CHROM,CENTER,WINDOW
2,1,114329037,TGAGATTTATATAATCTTGTACTCTCAGCACTTAGAGCAGAACAAG...
3,1,47398604,AGGACTACCGGTGTGCACCACCATGCCAGGCTAATTTTTGTATTTT...
4,1,166421804,GAGGGATTGTCAACATGAAAACCACCATGAAGAGGTAAGAATGGTT...
5,1,69215104,CAAAAGAAGGGGGGAGGTCTCTTCCTTTGTTTGCAATAGGGAGTAT...
6,1,29412419,CTGGACAGAAAAAGTGGAGTTTATGTGCAAAATTTATGTGCAAAGG...
...,...,...,...
177991,22,11487044,CTTCCCAAAGGTGGAAGCAGGAAGCGAGTTTTCTCAGGTGTTTGCC...
177993,22,27193525,TGACTTAAATAGTGAAGAGGAGACAGTTTGCTTCAATGCTTGGCAC...
177994,22,12064284,TAACTAAAGAAAATTAAGATAAAATAATTTTGACAACAGCTTGACC...
177995,22,29303607,CTCTCAGTCACCAACCACCTCTAATGTTGCCCAATCAATGGTCTTT...


In [182]:
# drop coordinates with windows containing bases other than 'A', 'C', 'G', 'T'

drop_invalid_windows(
	windows=intergen_nonGCmatch_windows,
	valid_bases=BASES_SET,
)

No windows dropped


In [826]:
# write out results

with open(f"{WINDOWS_DIR}/hg38_intergen_nonGCmatch.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_windows.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

# **3.** <u>Generate matches</u>

This section finds DNMs that fall into the windows ("matches"). This is done for all 3 sets of coordinates (TSS regions, GC matched intergenic, and non-GC matched intergenic).

## **3.1** TSS

This sub-section finds matches for the TSS coordinates. Matches are aggregatedc in a list (`tss_matches_list`) and subsequently loaded into a Pandas DataFrame (`tss_matches`).

Each match is packed with the following metrics:

|Metric|Description|
|-:|-|
|`CHROM`|chromosome of the DNM|
|`POS`|DNM position on chromosome CHROM|
|`REF`|base at position POS in reference genome|
|`ALT`|base found at position POS in proband|
|`TSS`|transcriptional start site coordinate|
|`FIRST_EIB`|coordinate of first extron-intron boundary|
|`DIST`|distance from TSS|
|`FIRST_EX`|*boolean*: within first exon?|
|`DOWNSTREAM`|*boolean*: downstream and after first exon?|
|`INTERGENIC`|*boolean*: within intergenic space immediately before TSS?|
|`STRAND`|strandedness|
|`ENSGID`|Ensebl gene ID|
|`ENSTID`|Ensebl transcript ID|
|`WINDOW`|string representing ±2501 base window centered around TSS|

In [115]:
POS_INDEX = 1   # column index for DNM position in dataset DataFrame

tss_matches_list = list()
for i, tss_coord in enumerate(tss_windows.values, start=1):
	chrom, tss, eib, strandedness, ensGID, ensTID, window = tss_coord
	
	# calculate window start and stop indices
	tss_window_start, tss_window_stop = tss - WINDOW_LENGTH//2, tss + WINDOW_LENGTH//2

	# filter for matches
	matches_df_temp = iceland_dataset[
		(iceland_dataset.POS >= tss_window_start) &
		(iceland_dataset.POS <  tss_window_stop) &
		(iceland_dataset.CHROM == chrom)
	]
	
	if len(matches_df_temp):
		# if matches exist, gather metrics for each

		match_data_temp = list()
		for item in matches_df_temp.values:
			if strandedness == '+':
				dist = item[POS_INDEX] - tss
				first_ex = int(item[POS_INDEX] <= eib & item[POS_INDEX] >= tss)
				downstream = int(item[POS_INDEX] >= eib)
				intergenic = int(item[POS_INDEX] < tss)

			elif strandedness == '-':
				dist = tss - item[POS_INDEX]
				first_ex = int(item[POS_INDEX] <= tss & item[POS_INDEX] >= eib)
				downstream = int(item[POS_INDEX] <= eib)
				intergenic = int(item[POS_INDEX] > tss)

			match_data_temp.append(
				[
					*item,	# contains CHROM, POS, REF, ALT
					tss,
					eib,
					dist,
					first_ex,
					downstream,
					intergenic,
					strandedness, 
					ensGID,
					ensTID,
					window,
				]
			)

		tss_matches_list.extend(match_data_temp)

	print(f"{i * 100 / len(tss_coords):.0f}%", end='\r')

print(f"{len(tss_matches_list) = }")

len(tss_matches_list) = 2950


In [116]:
tss_matches_columns = [	# column headers
	'CHROM',
	'POS',
	'REF',
	'ALT',
	'TSS',
	'FIRST_EIB',
	'DIST',
	'FIRST_EX',
	'DOWNSTREAM',
	'INTERGENIC',
	'STRAND',
	'ENSGID',
	'ENSTID',
	'WINDOW',
]

tss_matches = pd.DataFrame(
	tss_matches_list,
	columns=tss_matches_columns,
)

# ensure dtypes for CHROM, TSS, FIRST_EIB, and DIST
tss_matches.CHROM = tss_matches.CHROM.astype(str)
tss_matches.TSS = tss_matches.TSS.astype(int)
tss_matches.FIRST_EIB = tss_matches.FIRST_EIB.astype(int)
tss_matches.DIST = tss_matches.DIST.astype(int)

tss_matches

,CHROM,POS,REF,ALT,TSS,FIRST_EIB,DIST,FIRST_EX,DOWNSTREAM,INTERGENIC,STRAND,ENSGID,ENSTID,WINDOW
0,1,1014727,G,T,1013497,1013576,1230,0,1,0,+,ENSG00000187608,ENST00000649529,CCTCGGTCTCTCAAAGTGCTGGGATTACAGGTGAGCCACTGCGCCC...
1,1,1021242,C,T,1020120,1020373,1122,0,1,0,+,ENSG00000188157,ENST00000379370,ATCTGGCCCCACCCACATCCTGCTGATGGGTAGAGCCTAGTGGTCT...
2,1,1208352,G,A,1206571,1206385,-1781,0,0,1,-,ENSG00000186891,ENST00000328596,GGGGGCGTCGTATCCCTTCAGTCTCCGGCAGGACAGGGGGTAGGTC...
3,1,1360908,G,A,1358543,1358456,-2365,0,0,1,-,ENSG00000162576,ENST00000445648,ACAGAAGTACACGTACGTACACATACACAGACACAGAAGCGCACAC...
4,1,1398713,C,T,1399311,1399019,598,0,1,0,-,ENSG00000221978,ENST00000481223,GGTGGGACCGAGGGGAGTTCCCGACGCACGTGTCACGGGCGAAAGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945,X,151976628,G,A,151974676,151974570,-1952,0,0,1,-,ENSG00000102287,ENST00000370328,GAGACAGTTTGTGGTGTGTATCGACCAAAGTTGTGGGACCGACCGA...
2946,X,152735883,C,T,152733735,152733668,-2148,0,0,1,-,ENSG00000198930,ENST00000370287,GTTCGGGTCCCGGAGAGGAACCCGGAGTTCCGGAAGGAGTCCGAAC...
2947,X,152735883,C,T,152733757,152733859,2126,0,1,0,+,ENSG00000213401,ENST00000393869,TAATAAAACGTTTTTTATCACAAATATGCCATTAAAAGTGAAATCC...
2948,X,153727239,G,A,153724856,153726166,2383,0,1,0,+,ENSG00000101986,ENST00000218104,TTCATTTGCTTTTCCAGTTTCCTTTGTGTAGAAATCAGATTGTCTG...


In [45]:
# write out results

with open(f"{MATCHES_DIR}/tss_matches.csv", WRITING_MODE) as fout:
	tss_matches.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

## **3.2** Intergenic

This sub-section finds matches for the intergenic coordinates.

In [188]:
def get_intergen_matches(windows: pd.DataFrame, dataset: pd.DataFrame, win_len: int=None, print_progress: bool=True) -> list:
	'''
	Get DNMs ("matches") that fall within the windows specified by `windows`.

	## Parameters
	windows: Pandas.DataFrame
		Windows around specific coordinates.
	dataset: Pandas.DataFrame
		DNMs dataset.
	win_len: int, default = None
		Window length to be used to consider DNMs. Can be less than or equal 
		to length of the window sequence associated with each coordinate in `windows`. 
		If none is specified, defaults to length of window sequence from `windows`.
	print_progress: bool, default = True
		If true, will print progress as percentage of completion.
		
	## Returns
	matches: list
		List with each item containing the following information for each coordinate:

			- chromosome
			- DNM position
			- reference base
			- alternate base
			- coordinate for center of window
			- distance from center
			- window sequence
	'''

	if win_len == None:
		win_len = len(windows.WINDOW.iloc[0])

	matches = list()
	for i, coord in enumerate(windows.values):
		chrom, center, window = coord

		temp_matches = dataset[
			(dataset.CHROM == chrom) &
			(dataset.POS >= (center - win_len//2)) &
			(dataset.POS <= (center + win_len//2))
		]

		if len(temp_matches):
			matches.extend([ (chrom, pos, ref, alt, center, (pos - center), window)
									for chrom, pos, ref, alt
									in temp_matches.values ])

		if print_progress: print(f"{i * 100 / len(windows):.0f}%", end='\r')

	return matches

In [ ]:
intergen_match_columns = [
	'CHROM',
	'POS',
	'REF',
	'ALT',
	'CENTER',
	'DIST',
	'WINDOW',
]

### **3.2.1** GC matched

This sub-sub-section finds matches for the GC matched intergenic coordinates. Matches are aggregatedc in a list (`intergen_GCmatch_match_list`) and subsequently loaded into a Pandas DataFrame (`intergen_GCmatch_matches`).

Each match is packed with the following metrics:

|Metric|Description|
|-:|-|
|`CHROM`|chromosome of the DNM|
|`POS`|DNM position on chromosome CHROM|
|`REF`|base at position POS in reference genome|
|`ALT`|base found at position POS in proband|
|`CENTER`|coordinate for center of window|
|`DIST`|distance from CENTER|
|`WINDOW`|string representing ±2501 base window centered around CENTER|

*NOTE: `INTERGEN_GCMATCH_WINLEN` is used to generate these matches, which is different from `WINDOW_LENGTH`. The window sequence represented by `WINDOW` was generated using `WINDOW_LENGTH`, and thus extends beyond `INTERGEN_GCMATCH_WINLEN` bases from `CENTER`*

In [184]:
intergen_GCmatch_match = get_intergen_matches(
	windows=intergen_GCmatch_windows,
    dataset=iceland_dataset,
	win_len=INTERGEN_GCMATCH_WINLEN,
)
intergen_GCmatch_matches = pd.DataFrame(
	intergen_GCmatch_match,
	columns=intergen_match_columns,
)
intergen_GCmatch_matches

,CHROM,POS,REF,ALT,CENTER,DIST,WINDOW
0,1,69216275,A,T,69216254,21,CTATTACCATGAGAATAATGTGTCTGGAGCAACCTGTGACAGTTTG...
1,1,171081744,T,A,171081712,32,AACCCAAATGTCCAACAATGATAGACTGGATTAAGAAAATGTGGCA...
2,1,238429890,C,T,238429898,-8,TTTTGGCCGTTCTTGCAGGAGTGAGATGGTATCACATTGTGGTTTT...
3,1,5297395,C,A,5297374,21,AAAGCATGGTCTTAGTTGTATCTTTTCAATTCCCAGGCAAAAGAGT...
4,1,54914884,A,C,54914879,5,ATCTCTACCAAAAATACAAAAAATTAGCCGGGCATGGTGGTGGGAG...
...,...,...,...,...,...,...,...
67,6,17206664,G,A,17206686,-22,TGCTTCCATTTGATGTTTTGTTTTGTTTTTCTTGTAACTGAGTCTC...
68,6,44063154,G,A,44063164,-10,GGATTCCTATAAAGTAAGGATACCCGTGGACCCCACCTCACCTGGC...
69,1,218165004,C,G,218165029,-25,AATATGGGTATAAACCAAAAATGTTTTGGTTTTCTTCTGTTTAAAG...
70,7,63114344,G,C,63114327,17,TTTGTATTTTTTAGTAGACACAGGGTTTCACCGTGTTAACCAGGAA...


In [47]:
# write out results

with open(f"{MATCHES_DIR}/intergen_GCmatch_matches_100b.csv", WRITING_MODE) as fout:
	intergen_GCmatch_matches.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

### **3.2.2** non-GC matched

This sub-sub-section finds matches for the non-GC matched intergenic coordinates, employing the same method as 3.2.1 (see chart in 3.2.1 for summary of metrics). Matches are aggregatedc in a list (`intergen_nonGCmatch_match_list`) and subsequently loaded into a Pandas DataFrame (`intergen_nonGCmatch_matches`).

In [190]:
intergen_nonGCmatch_matches = get_intergen_matches(
	windows=intergen_nonGCmatch_windows,
    dataset=iceland_dataset,
	win_len=WINDOW_LENGTH,
)
intergen_nonGCmatch_matches = pd.DataFrame(
	intergen_nonGCmatch_matches,
	columns=intergen_match_columns,
)
intergen_nonGCmatch_matches

,CHROM,POS,REF,ALT,CENTER,DIST,WINDOW
0,1,69216275,A,T,69215104,1171,CAAAAGAAGGGGGGAGGTCTCTTCCTTTGTTTGCAATAGGGAGTAT...
1,1,171476920,T,A,171475608,1312,CACAGTTCCACATGACTGGAGAGGCCTCACAATCATGGCAGAAGGC...
2,1,171081744,T,A,171081362,382,AAAATGCTCACCATCACTGGCCATCAGAGAAATGCAAATCAAAACC...
3,1,192455849,A,C,192455397,452,TAACTTCACTACCTGATAGCTCTGTGATCTTGGATATGTTTCTAAA...
4,1,238429890,C,T,238427448,2442,ACTTACATGTGAAAAAAATCTTGAAATTATATTTTGAATTTTCATT...
...,...,...,...,...,...,...,...
28869,22,23675643,G,A,23673219,2424,GGGCAAGGAGCAGGGTCAGACCCAGCCTCAGACACTCCCTGGCACC...
28870,22,34300244,A,G,34300075,169,CAGAGCCCAGAAGAAGAACTGCTGAGCTCTGAGTCCTGGACTTTTT...
28871,22,47662694,G,A,47660885,1809,CCCTCCACCAGGAATGCCTTTGCCCCTTTGTGCATGGCCAGGTCCA...
28872,22,29305002,G,A,29303607,1395,CTCTCAGTCACCAACCACCTCTAATGTTGCCCAATCAATGGTCTTT...


In [51]:
# write out results

with open(f"{MATCHES_DIR}/intergen_nonGCmatch_matches.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_matches.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

______

# **4.** <u>Separate CpG and non-CpG</u>

This section separates CpG and non-CpG DNMs.

In [195]:
def separate_CpG_matches(matches: pd.DataFrame, ref_idx: int, alt_idx: int, dist_idx: int, win_len: int, window_idx: int=-1, offset: int=0) -> tuple:
    '''
    Separate CpG and non-CpG DNMs.

    ## Parameters
    matches: Pandas.DataFrame
        DNM DataFrame.
    ref_idx: int
        REF column index.
    alt_idx: int
        ALT column index.
    dist_idx: int
        DIST column index.
    win_len: int
        Window length to be used to consider DNMs. Can be less than or equal 
        to length of the window sequence associated with each match in `matches`.
    window_idx: int, default = -1
        WINDOW column index.
    offset: int, default = 0
        Optional offset for positional indexing within the window.
        
    ## Returns
    separated: tuple
        Tuple containing two Pandas.DataFrames, the first being the CpG DNMs and 
        the second the non-CpG DNMs.
    '''

    CpG_matches_list, nonCpG_matches_list = list(), list()
    for dnm in matches.values:
        ref = dnm[ref_idx]
        alt = dnm[alt_idx]
        dist = dnm[dist_idx]
        window = dnm[window_idx]

        if   ref == 'C' and alt == 'T' and window[win_len//2+int(dist)+1+offset] == 'G':   # CpG C>T
            CpG_matches_list.append(dnm)
        elif ref == 'G' and alt == 'A' and window[win_len//2+int(dist)-1+offset] == 'C':   # CpG G>A
            CpG_matches_list.append(dnm)
        else: nonCpG_matches_list.append(dnm)

    CpG_matches = pd.DataFrame(CpG_matches_list, columns=matches.columns)
    nonCpG_matches = pd.DataFrame(nonCpG_matches_list, columns=matches.columns)

    return CpG_matches, nonCpG_matches

## **4.1** TSS

This sub-section separates CpG and non-CpG DNMs around the TSS into 2 Pandas DataFrames:
- `tss_CpG`: CpG DNMs around the TSS
- `tss_nonCpG`: non-CpG DNMs around the TSS

In [194]:
tss_CpG, tss_nonCpG = separate_CpG_matches(
    matches=tss_matches,
    ref_idx=2,
    alt_idx=3,
    dist_idx=6,
    win_len=WINDOW_LENGTH,
    offset=CPG_OFFSET,
)
print(f"{len(tss_CpG) = }, {len(tss_nonCpG) = }")

len(tss_CpG) = 426, len(tss_nonCpG) = 2524


In [53]:
# write out results

with open(f"{MATCHES_DIR}/tss_CpG.csv", WRITING_MODE) as fout:
	tss_CpG.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

with open(f"{MATCHES_DIR}/tss_nonCpG.csv", WRITING_MODE) as fout:
	tss_nonCpG.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

## **4.2** Intergenic

This sub-section separactes CpG and non-CpG DNMs around the intergenic coordinates.

### **4.2.1** GC matched

This sub-sub-section separactes CpG and non-CpG DNMs around the intergenic GC matched coordinates into 2 Pandas DataFrames:
- `intergen_GCmatch_CpG`: CpG DNMs around the intergenic GC matched coordinates
- `intergen_GCmatch_nonCpG`: non-CpG DNMs around the intergenic GC matched coordinates

In [196]:
intergen_GCmatch_CpG, intergen_GCmatch_nonCpG = separate_CpG_matches(
    matches=intergen_GCmatch_matches,
    ref_idx=2,
    alt_idx=3,
    dist_idx=5,
    win_len=WINDOW_LENGTH,
)
print(f"{len(intergen_GCmatch_CpG) = }, {len(intergen_GCmatch_nonCpG) = }")

len(intergen_GCmatch_CpG) = 32, len(intergen_GCmatch_nonCpG) = 40


In [56]:
# write out results

with open(f"{MATCHES_DIR}/intergen_GCmatch_CpG_100b.csv", WRITING_MODE) as fout:
	intergen_GCmatch_CpG.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

with open(f"{MATCHES_DIR}/intergen_GCmatch_nonCpG_100b.csv", WRITING_MODE) as fout:
	intergen_GCmatch_nonCpG.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

### **4.2.2** non-GC matched

This sub-sub-section separactes CpG and non-CpG DNMs around the intergenic non-GC matched coordinates into 2 Pandas DataFrames:
- `intergen_nonGCmatch_CpG`: CpG DNMs around the intergenic non-GC matched coordinates
- `intergen_nonGCmatch_nonCpG`: non-CpG DNMs around the intergenic non-GC matched coordinates

In [197]:
intergen_nonGCmatch_CpG, intergen_nonGCmatch_nonCpG = separate_CpG_matches(
    matches=intergen_nonGCmatch_matches,
    ref_idx=2,
    alt_idx=3,
    dist_idx=5,
    win_len=WINDOW_LENGTH,
)
print(f"{len(intergen_nonGCmatch_CpG) = }, {len(intergen_nonGCmatch_nonCpG) = }")

len(intergen_nonGCmatch_CpG) = 4748, len(intergen_nonGCmatch_nonCpG) = 24126


In [58]:
# write out results

with open(f"{MATCHES_DIR}/intergen_nonGCmatch_CpG.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_CpG.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

with open(f"{MATCHES_DIR}/intergen_nonGCmatch_nonCpG.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_nonCpG.to_csv(
		fout,
		index=False,
		lineterminator='\n',
	)

________

# **5.** <u>Positional counts</u>

This section generates positional counts for each base. That is, this counts the number of occurances of each base in each unique position in the ±2501 base window.

Each subsection generates 3 sets of positional counts:
- total positional counts (CpG + non-CpG)
- positional counts excluding CpGs
- C and G positional counts within CpGs

In [198]:
def get_total_pos_counts(windows: pd.DataFrame, win_len: int, offset: int=0, column_headers: list[str]=None) -> pd.DataFrame:
    '''
    Get positional counts for each base within windows specified by `windows`.

    ## Parameters
    windows: Pandas.DataFrame
        Windows around coordinates.
    win_len: int
        Length of window.
    offset: int, default = 0
        Window offset.
    column_headers: list, default = None
        If none is specified, defaults to ['A','C','G','T'].
        
    ## Returns
    matches: Pandas.DataFrame
        DataFrame containing positional counts for each base. 
        Rows are position relative to center of window, and 
        columns are the bases.
    '''

    if column_headers == None:
        column_headers = ['A','C','G','T']

    # grab windows from coordinate DataFrame
    windows_listed = np.array([ list(window[offset:-offset])
                                    for window
                                    in windows.WINDOW ]).transpose()

    # count unique occurences of each base
    pos_counts = np.array([ np.unique(row, return_counts=True)[1]
                                for row
                                in windows_listed ])

    return pd.DataFrame(
        pos_counts,
        columns=column_headers,
        index=range(-win_len//2,win_len//2),
    )

In [206]:
def get_CpG_pos_counts(windows: pd.DataFrame, win_len: int, CpG_str: str='CG', offset: int=1, column_headers: list[str]=None) -> pd.DataFrame:
	'''
	Get positional counts for each CpG within windows specified by `windows`.

	## Parameters
	windows: Pandas.DataFrame
		Windows around coordinates.
	win_len: int
		Length of window.
	CpG_str: str, default = 'CG'
		String representation of a CpG.
	offset: int, default = 1
		Offsets C counts and G counts, relative to each other.
	column_headers: list, default = None
		If none is specified, defaults to ['A','C','G','T'].
		
	## Returns
	CpG_pos_counts: Pandas.DataFrame
		DataFrame containing positional counts for each base. 
		Rows are position relative to center of window, and 
		columns are the bases.
	'''

	if column_headers == None:
		column_headers = ['A','C','G','T']

	# grab CpGs (used for non-CpG and CpG positional counts)
	CpG_pos_counts_raw = [ [CpG_match.start() for CpG_match in re.finditer(CpG_str, i)]
								for i
								in windows.WINDOW.iloc ]

	# flatten raw counts
	CpG_pos_counts_flattened = np.array([ num
											for item
											in CpG_pos_counts_raw
											for num in (item if isinstance(item, list) else (item,)) ])

	# offset raw counts
	CpG_pos_counts_flattened -= (win_len//2 + 1)


	CpG_pos_counts_dict = OrderedDict(sorted(
		Counter(CpG_pos_counts_flattened).items()
	))

	return pd.DataFrame(
		[
			np.zeros(win_len),										# 'A' count (zero)
			np.array([*CpG_pos_counts_dict.values(),][offset:]),	# 'C' count
			np.array([*CpG_pos_counts_dict.values(),][:-offset]),	# 'G' count
			np.zeros(win_len),										# 'T' count (zero)
		],
		columns=range(-win_len//2,win_len//2),
		index=column_headers,
		dtype=int,
	).T

In [219]:
def get_nonCpG_pos_counts(total_pos_counts: pd.DataFrame, CpG_pos_counts: pd.DataFrame, offset: int=1, column_headers: list[str]=None) -> pd.DataFrame:
	'''
	Get positional counts for each non-CpG base within windows specified by `windows`.

	## Parameters
	total_pos_counts: Pandas.DataFrame
		Total positional counts.
	CpG_pos_counts: Pandas.DataFrame
		CpG positional counts.
	offset: int, default = 1
		Offsets C counts and G counts, relative to each other.
	column_headers: list, default = None
		If none is specified, defaults to ['A','C','G','T'].
		
	## Returns
	nonCpG_pos_counts: Pandas.DataFrame
		DataFrame containing positional counts for each base. 
		Rows are position relative to center of window, and 
		columns are the bases.
	'''

	if column_headers == None:
		column_headers = ['A','C','G','T']

	nonCpG_C_pos_counts = total_pos_counts.C - np.array([np.zeros(offset),*CpG_pos_counts.C][offset:])
	nonCpG_G_pos_counts = total_pos_counts.G - np.array([*CpG_pos_counts.G,np.zeros(offset)][:-offset])

	return pd.DataFrame(
		[
			total_pos_counts['A'],	# 'A' count
			nonCpG_C_pos_counts,	# 'C' count
			nonCpG_G_pos_counts,	# 'G' count
			total_pos_counts['T'],	# 'T' count
		],
		columns=total_pos_counts.index,
		index=total_pos_counts.columns,
		dtype=int,
	).T

## **5.1** TSS

This sub-section generates positional counts for the TSS coordinates.

In [199]:
# total positional counts

tss_pos_counts = get_total_pos_counts(
    windows=tss_windows,
    win_len=WINDOW_LENGTH,
    offset=CPG_OFFSET,
)
tss_pos_counts

,A,C,G,T
-2500,4653,3802,3865,4784
-2499,4717,3855,3913,4619
-2498,4674,3818,3894,4718
-2497,4753,3840,3870,4641
-2496,4746,3872,3880,4606
...,...,...,...,...
2495,4648,3827,3919,4710
2496,4660,3874,3894,4676
2497,4594,3931,3864,4715
2498,4720,3828,3793,4763


In [207]:
# CpG positional counts

tss_CpG_pos_counts = get_CpG_pos_counts(
	windows=tss_windows,
	win_len=WINDOW_LENGTH,
)
tss_CpG_pos_counts

,A,C,G,T
-2500,0,553,537,0
-2499,0,534,553,0
-2498,0,563,534,0
-2497,0,594,563,0
-2496,0,551,594,0
...,...,...,...,...
2495,0,562,535,0
2496,0,546,562,0
2497,0,519,546,0
2498,0,549,519,0


In [220]:
# non-CpG positional counts

tss_nonCpG_pos_counts = get_nonCpG_pos_counts(
	total_pos_counts=tss_pos_counts,
	CpG_pos_counts=tss_CpG_pos_counts,
)
tss_nonCpG_pos_counts

,A,C,G,T
-2500,4653,3249,3328,4784
-2499,4717,3321,3360,4619
-2498,4674,3255,3360,4718
-2497,4753,3246,3307,4641
-2496,4746,3321,3286,4606
...,...,...,...,...
2495,4648,3265,3384,4710
2496,4660,3328,3332,4676
2497,4594,3412,3318,4715
2498,4720,3279,3274,4763


In [63]:
# write out results

with open(f"{POS_COUNTS_SUBDIR}/tss_pos_counts.csv", WRITING_MODE) as fout:
	tss_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)
with open(f"{POS_COUNTS_SUBDIR}/tss_nonCpG_pos_counts.csv", WRITING_MODE) as fout:
	tss_nonCpG_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)
with open(f"{POS_COUNTS_SUBDIR}/tss_CpG_pos_counts.csv", WRITING_MODE) as fout:
	tss_CpG_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)

## **5.2** Intergenic

This sub-section generates positional counts for the intergenic coordinates.

### **5.2.1** GC matched

This sub-sub-section generates positional counts for the intergenic GC matched coordinates.

In [200]:
# total positional counts

intergen_GCmatch_pos_counts = get_total_pos_counts(
    windows=intergen_GCmatch_windows,
    win_len=WINDOW_LENGTH,
    offset=CPG_OFFSET,
)
intergen_GCmatch_pos_counts

,A,C,G,T
-2500,4601,3761,3769,4437
-2499,4612,3732,3797,4427
-2498,4537,3809,3746,4476
-2497,4519,3743,3771,4535
-2496,4418,3738,3948,4464
...,...,...,...,...
2495,4523,3842,3727,4476
2496,4412,3770,3735,4651
2497,4343,3813,3795,4617
2498,4533,3861,3783,4391


In [221]:
# CpG positional counts

intergen_GCmatch_CpG_pos_counts = get_CpG_pos_counts(
	windows=intergen_GCmatch_windows,
	win_len=WINDOW_LENGTH,
)
intergen_GCmatch_CpG_pos_counts

,A,C,G,T
-2500,0,250,236,0
-2499,0,240,250,0
-2498,0,237,240,0
-2497,0,259,237,0
-2496,0,247,259,0
...,...,...,...,...
2495,0,247,230,0
2496,0,239,247,0
2497,0,235,239,0
2498,0,246,235,0


In [222]:
# non-CpG positional counts

intergen_GCmatch_nonCpG_pos_counts = get_nonCpG_pos_counts(
	total_pos_counts=intergen_GCmatch_pos_counts,
	CpG_pos_counts=intergen_GCmatch_CpG_pos_counts,
)
intergen_GCmatch_nonCpG_pos_counts

,A,C,G,T
-2500,4601,3511,3533,4437
-2499,4612,3492,3547,4427
-2498,4537,3572,3506,4476
-2497,4519,3484,3534,4535
-2496,4418,3491,3689,4464
...,...,...,...,...
2495,4523,3595,3497,4476
2496,4412,3531,3488,4651
2497,4343,3578,3556,4617
2498,4533,3615,3548,4391


In [68]:
# write out results

with open(f"{POS_COUNTS_SUBDIR}/intergen_GCmatch_pos_counts.csv", WRITING_MODE) as fout:
	intergen_GCmatch_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)
with open(f"{POS_COUNTS_SUBDIR}/intergen_GCmatch_nonCpG_pos_counts.csv", WRITING_MODE) as fout:
	intergen_GCmatch_nonCpG_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)
with open(f"{POS_COUNTS_SUBDIR}/intergen_GCmatch_CpG_pos_counts.csv", WRITING_MODE) as fout:
	intergen_GCmatch_CpG_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)

### **5.2.2** non-GC matched

This sub-sub-section generates positional counts for the intergenic GC matched coordinates.

In [201]:
# total positional counts

intergen_nonGCmatch_pos_counts = get_total_pos_counts(
    windows=intergen_nonGCmatch_windows,
    win_len=WINDOW_LENGTH,
    offset=CPG_OFFSET,
)
intergen_nonGCmatch_pos_counts

,A,C,G,T
-2500,49224,32580,33004,48700
-2499,48765,32642,33171,48930
-2498,48774,32637,32874,49223
-2497,49072,32798,32623,49015
-2496,48686,32994,33049,48779
...,...,...,...,...
2495,48612,32497,33069,49330
2496,49170,33018,32741,48579
2497,48749,32576,32812,49371
2498,48809,32529,32996,49174


In [223]:
# CpG positional counts

intergen_nonGCmatch_CpG_pos_counts = get_CpG_pos_counts(
	windows=intergen_nonGCmatch_windows,
	win_len=WINDOW_LENGTH,
)
intergen_nonGCmatch_CpG_pos_counts

,A,C,G,T
-2500,0,1544,1493,0
-2499,0,1434,1544,0
-2498,0,1472,1434,0
-2497,0,1517,1472,0
-2496,0,1553,1517,0
...,...,...,...,...
2495,0,1469,1552,0
2496,0,1454,1469,0
2497,0,1440,1454,0
2498,0,1469,1440,0


In [224]:
# non-CpG positional counts

intergen_nonGCmatch_nonCpG_pos_counts = get_nonCpG_pos_counts(
	total_pos_counts=intergen_nonGCmatch_pos_counts,
	CpG_pos_counts=intergen_nonGCmatch_CpG_pos_counts,
)
intergen_nonGCmatch_nonCpG_pos_counts

,A,C,G,T
-2500,49224,31036,31511,48700
-2499,48765,31208,31627,48930
-2498,48774,31165,31440,49223
-2497,49072,31281,31151,49015
-2496,48686,31441,31532,48779
...,...,...,...,...
2495,48612,31028,31517,49330
2496,49170,31564,31272,48579
2497,48749,31136,31358,49371
2498,48809,31060,31556,49174


In [73]:
# write out results

with open(f"{POS_COUNTS_SUBDIR}/intergen_nonGCmatch_pos_counts.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)
with open(f"{POS_COUNTS_SUBDIR}/intergen_nonGCmatch_nonCpG_pos_counts.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_nonCpG_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)
with open(f"{POS_COUNTS_SUBDIR}/intergen_nonGCmatch_CpG_pos_counts.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_CpG_pos_counts.to_csv(
		fout,
		lineterminator='\n',
	)

____

# **6.** <u>Regional counts (TSS only)</u>

In [248]:
tss_regional_counts_regions = [
	'INTERGEN',
	'FIRST_EX',
	'DOWNSTREAM',
]

## **6.1** Count instances of mutable bases in each region

In [247]:
# generate dict of bases and counts for each region
tss_counts = [ dict(zip(BASES_SET, np.zeros(len(BASES_SET), dtype=int)))
			  	for i
				in range(len(tss_regional_counts_regions)) ]

# instantiate global CpG counters for each region
intergen_CpG_count, first_ex_CpG_count, downstream_CpG_count = 0, 0, 0


for match in tss_windows.values:
	_, tss, eib, _, _, _, window = match

	# calculate positional index for exon-intron boundary
	eib_index = WINDOW_LENGTH//2 + abs(eib - tss)
	
	# get sequence for each region
	intergenic_seq = window[ : WINDOW_LENGTH//2 + 1 + CPG_OFFSET]
	first_ex_seq = window[WINDOW_LENGTH//2 - 1 + CPG_OFFSET : eib_index + 1 + CPG_OFFSET]
	downstream_seq = window[eib_index - 1 + CPG_OFFSET : ]

	# count CpGs in each region
	intergen_CpG_count += len(re.findall(CPG_STR,intergenic_seq))
	first_ex_CpG_count += len(re.findall(CPG_STR,first_ex_seq))
	downstream_CpG_count += len(re.findall(CPG_STR,downstream_seq))

	# count number of mutable bases in each region
	counts_temp = [ Counter(seq[CPG_OFFSET:-CPG_OFFSET])
						for seq
						in (intergenic_seq, first_ex_seq, downstream_seq) ]

	# store mutable bases counts in global counter
	for i, counts in enumerate(counts_temp):
		for base, base_count in counts.items():
			tss_counts[i][base] += base_count


tss_region_counts = pd.DataFrame(
	tss_counts,
	index=tss_regional_counts_regions,
)
tss_region_counts = tss_region_counts[['A','T','C','G']]	# explicitly reorder

tss_region_counts

,A,T,C,G
INTERGEN,10884639,10894242,10495883,10485236
FIRST_EX,915616,910596,1620611,1622955
DOWNSTREAM,9530273,9570171,9294443,9295335


In [246]:
CpG_region_counts = intergen_CpG_count, first_ex_CpG_count, downstream_CpG_count

# simple aggregation of regional CpG counts
tss_CpG_region_counts = pd.DataFrame(
	[
        np.zeros(len(tss_regional_counts_regions), dtype=int),	# A
        np.zeros(len(tss_regional_counts_regions), dtype=int),	# T
        CpG_region_counts,										# C
        CpG_region_counts,										# G
    ],
	columns=tss_region_counts.index,
	index=tss_region_counts.columns,
).T

tss_CpG_region_counts

,A,T,C,G
INTERGEN,0,0,1896839,1896839
FIRST_EX,0,0,492596,492596
DOWNSTREAM,0,0,1616800,1616800


In [137]:
# subtract regional CpG counts from regional mutable C and G regional counts
nonCpG_CG_region_counts = [	[tss_counts[i][base] - CpG_region_counts[i] for i in range(len(tss_regional_counts_regions))]
								for base
								in CPG_STR ]

nonCpG_region_counts = [
	tss_region_counts['A'].to_list(),	# A
	tss_region_counts['T'].to_list(),	# T
	nonCpG_CG_region_counts[0],			# C
	nonCpG_CG_region_counts[1],			# G
]

tss_nonCpG_region_counts = pd.DataFrame(
	nonCpG_region_counts,
	columns=tss_regional_counts_regions,
	index=tss_region_counts.columns,
).T
tss_nonCpG_region_counts

,A,T,C,G
INTERGEN,10884639,10894242,8599044,8588397
FIRST_EX,915616,910596,1128015,1130359
DOWNSTREAM,9530273,9570171,7677643,7678535


In [77]:
# write out results

with open(f"{COUNTS_DIR}/regional/tss_region_counts.csv", WRITING_MODE) as fout:
	tss_region_counts.to_csv(
		fout,
		lineterminator='\n',
	)

with open(f"{COUNTS_DIR}/regional/tss_nonCpG_region_counts.csv", WRITING_MODE) as fout:
	tss_nonCpG_region_counts.to_csv(
		fout,
		lineterminator='\n',
	)

with open(f"{COUNTS_DIR}/regional/tss_CpG_region_counts.csv", WRITING_MODE) as fout:
	tss_CpG_region_counts.to_csv(
		fout,
		lineterminator='\n',
	)

## **6.2** Count instances of unique mutation type in each region

In [237]:
def get_regional_dnm_type_counts(matches: pd.DataFrame, dnm_types: set[str], regions: list[str]) -> pd.DataFrame:
	'''
	Get regional DNM type counts for each DNM type (i.e. 'C>T', 'A>C') in `matches`.

	## Parameters
	matches: Pandas.DataFrame
		DataFrame containing DNM dataset.
	dnm_types: set
		Set of all possible DNM types.
	regions: list
		List of strings representing region names.
	## Returns
	counts: Pandas.DataFrame
		DataFrame containing regional counts for each DNM type. 
		Rows are regions, and columns are the DNM types.
	'''

	counts = [ get_ordered_counts_for_region(matches=matches, dnm_types=dnm_types, region=region)
				for region
				in regions ]

	return pd.DataFrame(
		counts,
		index=regions,
	)

def get_ordered_counts_for_region(matches: pd.DataFrame, region: str, dnm_types: set[str]) -> OrderedDict:
	'''
	Get DNM type counts for each DNM type in the region specified by `region`.

	## Parameters
	matches: Pandas.DataFrame
		DataFrame containing DNM dataset.
	region: str
		Strings representing region name.
	dnm_types: set
		Set of all possible DNM types.
	## Returns
	ordered_counts: OrderedDict
		OrderedDict containing all possible DNM types 
		and their respective counts for the region.
	'''

	counts = Counter([ '>'.join(ref_alt)
						for ref_alt
						in matches[matches[region].astype(int) == 1][['REF','ALT']].values ])
	set_zero_values(counts=counts, dnm_types=dnm_types)

	return OrderedDict(sorted(counts.items()))

def set_zero_values(counts: Counter, dnm_types: set[str]) -> None:
	'''
	Adds missing DNM types to `counts`, and sets their 
	respective count to zero.

	## Parameters
	counts: Counter
		Counter containing DNM types and their respective counts.
	dnm_types: set
		Set of all possible DNM types.
	'''

	counts.update({ dnm_type : 0
						for dnm_type
						in dnm_types - set(counts.keys()) })


### **6.2.1** Total (non-CpG + CpG)

In [238]:
tss_dnmtype_counts = get_regional_dnm_type_counts(
	matches=tss_matches,
	dnm_types=DNM_TYPES,
	regions=tss_regional_counts_regions,
)
tss_dnmtype_counts

,A>C,A>G,A>T,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G
INTERGENIC,41,141,36,48,96,331,340,87,52,39,149,36
FIRST_EX,0,2,1,1,4,6,6,0,0,0,4,0
DOWNSTREAM,34,146,36,66,80,301,300,107,56,30,183,44


In [79]:
# write out results

with open(f"{COUNTS_DIR}/tss_dnmtype_counts.csv", WRITING_MODE) as fout:
	tss_dnmtype_counts.to_csv(
		fout,
		lineterminator='\n',
	)

### **6.2.2** non-CpG

In [239]:
tss_nonCpG_dnmtype_counts = get_regional_dnm_type_counts(
	matches=tss_nonCpG,
	dnm_types=DNM_TYPES,
	regions=tss_regional_counts_regions,
)
tss_nonCpG_dnmtype_counts

,A>C,A>G,A>T,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G
INTERGENIC,41,141,36,48,96,224,238,87,52,39,149,36
FIRST_EX,0,2,1,1,4,1,6,0,0,0,4,0
DOWNSTREAM,34,146,36,66,80,192,221,107,56,30,183,44


In [81]:
# write out results

with open(f"{COUNTS_DIR}/tss_nonCpG_dnmtype_counts.csv", WRITING_MODE) as fout:
	tss_nonCpG_dnmtype_counts.to_csv(
		fout,
		lineterminator='\n',
	)

### **6.2.2** CpG

*Note: this is a redundant calculation as these values can be obtained by simply subtracting the non-CpG counts from the total counts. This redundant calculation is done to verify the previous results.*

In [240]:
tss_CpG_dnmtype_counts = get_regional_dnm_type_counts(
	matches=tss_CpG,
	dnm_types=DNM_TYPES,
	regions=tss_regional_counts_regions,
)
tss_CpG_dnmtype_counts

,A>C,A>G,A>T,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G
INTERGENIC,0,0,0,0,0,107,102,0,0,0,0,0
FIRST_EX,0,0,0,0,0,5,0,0,0,0,0,0
DOWNSTREAM,0,0,0,0,0,109,79,0,0,0,0,0


In [83]:
# write out results

with open(f"{COUNTS_DIR}/tss_CpG_dnmtype_counts.csv", WRITING_MODE) as fout:
	tss_CpG_dnmtype_counts.to_csv(
		fout,
		lineterminator='\n',
	)

___________

# **7.** <u>Positional Metrics</u>

In [298]:
def print_positional_metrics(df: pd.DataFrame, total_pos_counts: pd.DataFrame, nonCpG_pos_counts:pd.DataFrame, win_len: int, sliding_win_len: int, GC_loss_dnm_types: list, include_CpG: bool, column_headers: list, parent_dir: str, writing_mode: str='x') -> None:
    '''
    Print to a CSV file the positional DNM type metrics for DNM types involving the loss or gain of GC content (i.e. 'C>T', 'A>G').

    Each set of metrics includes:
    |Metric|Description|
    |-|-|
    |Number of DNMs|Number of occurences of the DNM type in a given position|
    |Average of DNMs|Sliding window average for number of the DNM|
    |Positional count|Positional count of mutable bases (i.e. REF)|
    |Rate of mutation|Positional rate of the DNM|
    |Average rate of mutation|Sliding window average for rate of mutation|

    Filenames will have the structure "ref" + "alt", where "ref" is the reference base 
    in a given DNM and "alt" is the base found in a proband. For example, for A>G DNMs, 
    the filename will be "AG.csv".

    ## Parameters
    df: Pandas.DataFrame
        Pandas DataFrame containing DNMs dataset.
    total_pos_counts: Pandas.DataFrame
        Pandas DataFrame containing total positional base counts (CpG + nonCpG).
    nonCpG_pos_counts: Pandas.DataFrame
        Pandas DataFrame containing nonCpG positional base counts.
    win_len: int
        Length of sequence window.
    sliding_win_len: int
        Length of sliding window used to generate Average of DNMs and Average rate of mutation.
    GC_loss_dnm_types: set
        Set of possible GC DNM types.
    include_CpG: bool
        Whether to include CpG DNMs in counts.
    column_headers: list
        Headers used for printed columns.
    parent_dir: str
        Parent directory to write output files.
    writing_mode: str, default = 'x'
        Whether to overwrite existing files. Default is 'x', which does not overwrite existing files. 
        If set to 'w', will overwrite existing files.
    '''

    for ref, alt in GC_loss_dnm_types:
        dnm_counts_array = get_dnm_counts(
            df=df,
            ref=ref,
            alt=alt,
            win_len=win_len,
        )

        selected_pos_counts = select_pos_counts(
            ref=ref,
            alt=alt,
            include_CpG=include_CpG,
            total_pos_counts=total_pos_counts,
            nonCpG_pos_counts=nonCpG_pos_counts,
        )

        avg_dnm_counts_array = sliding_window_avg(
            arr=dnm_counts_array,
            sliding_win_len=sliding_win_len,
        )

        mut_rate_array = dnm_counts_array / selected_pos_counts

        avg_mut_rate_array = sliding_window_avg(
            arr=mut_rate_array,
            sliding_win_len=sliding_win_len,
        )

        df_tmp = pd.DataFrame(
            data = [
                dnm_counts_array,
                avg_dnm_counts_array,
                selected_pos_counts,
                mut_rate_array,
                avg_mut_rate_array,
            ],
        ).T.convert_dtypes()
        df_tmp.columns = column_headers
        df_tmp.index = range(-win_len//2, win_len//2)

        if not os.path.exists(parent_dir):
            os.makedirs(parent_dir)

        # write out
        with open(f"{parent_dir}/{ref}{alt}.csv", writing_mode) as fout:
            df_tmp.to_csv(
                fout,
                float_format='%.10f',
                lineterminator='\n',
            )


def print_positional_netGC(df: pd.DataFrame, pos_counts:pd.DataFrame, win_len: int, GC_loss_dnm_types: list, column_headers: list, parent_dir: str, sliding_win_len: int=100, truncated_output: bool=False, writing_mode: str='x') -> None:
	'''
	Print to a CSV file the net GC metrics for DNM types involving the loss or gain of GC content (i.e. 'C>T', 'A>G').

	If `truncated_output` is `False`, each set of metrics includes:
	|Metric|Description|
	|-|-|
	|Number of DNMs|Net number of occurences of the DNM type in a given position|
	|Average of DNMs|Sliding window average for net number of the DNM|
	|Positional count|Positional count of C and G|
	|Rate of mutation|Net positional rate of the DNM|
	|Average rate of mutation|Sliding window average for net positional rate of mutation|

	Filename will be "netGC.csv".

	If `truncated_output` is `False`, each set of metrics includes:
	|Metric|Description|
	|-|-|
	|Net GC change|Net positional change in GC|
	|Positional count|Positional count of C and G|

	Filename will be "netGC_#b.csv", where "#" is the window size specified in `win_len`.

	## Parameters
	df: Pandas.DataFrame
		Pandas DataFrame containing DNMs dataset.
	pos_counts: Pandas.DataFrame
		Pandas DataFrame containing positional base counts.
	win_len: int
		Length of sequence window.
	GC_loss_dnm_types: set
		Set of possible GC DNM types.
	column_headers: list
		Headers used for printed columns.
	parent_dir: str
		Parent directory to write output files.
	sliding_win_len: int, default = 100
		Length of sliding window used to generate Average of DNMs and Average rate of mutation.
	truncated_output: bool, default = False
		Whether output will be truncated.
	writing_mode: str, default = 'x'
		Whether to overwrite existing files. Default is 'x', which does not overwrite existing files. 
		If set to 'w', will overwrite existing files.
	'''

    
	# get positional DNM counts for all GC gain/loss DNM types
	raw_counts = [ get_dnm_counts(df=df, ref=ref, alt=alt, win_len=win_len)
					for ref, alt
					in GC_loss_dnm_types ]

	# calculate net GC change
	#                                   net GC gain                                                       net GC loss
	net_CG = (raw_counts[0] + raw_counts[1] + raw_counts[2] + raw_counts[3]) - (raw_counts[4] + raw_counts[5] + raw_counts[6] + raw_counts[7])

	if not truncated_output:
		pos_counts_C = pos_counts.C.to_numpy()
		pos_counts_G = pos_counts.G.to_numpy()

		# get average GC change using sliding window averaging
		avg_CG = sliding_window_avg(
			arr=net_CG,
			sliding_win_len=sliding_win_len,
		)

		# calculate net GC change rate
		CG_rate = net_CG / (pos_counts_C + pos_counts_G)

		# calculate average GC change rate using sliding window averaging
		avg_CG_rate = sliding_window_avg(
			arr=CG_rate,
			sliding_win_len=sliding_win_len,
		)

		data = [
			net_CG,
			avg_CG,
			(pos_counts_C + pos_counts_G),
			CG_rate,
			avg_CG_rate,
		]

	else:
		positional_GC_count = pos_counts.C.loc[-win_len//2 : win_len//2-1].to_numpy() + pos_counts.G.loc[-win_len//2 : win_len//2-1].to_numpy()
		data = [
			net_CG,
			positional_GC_count,
		]

	df_tmp = pd.DataFrame(data).T.convert_dtypes()
	df_tmp.columns = column_headers
	df_tmp.index = range(-win_len//2, win_len//2)

	if not os.path.exists(parent_dir):
		os.makedirs(parent_dir)

	fname = "netGC" if not truncated_output else f"netGC_{win_len}b"
	with open(f"{parent_dir}/{fname}.csv", writing_mode) as fout:
		df_tmp.to_csv(
			fout,
			float_format='%.10f',
			lineterminator='\n',
		)


def select_pos_counts(ref: str, alt: str, include_CpG: bool, total_pos_counts: pd.DataFrame, nonCpG_pos_counts: pd.DataFrame) -> np.ndarray:
    '''
    Selects a counts for `ref`, based on `ref` and `alt`, or `include_CpG`.

    If `include_CpG` is `True`, will always return `ref` from `total_pos_counts`. Use this for when CpG DNMs
    are to be included in positional metrics.
    
    If `include_CpG` is `False`, `ref` will be returned from `nonCpG_pos_counts` when `ref` is 'C' and `alt` 
    is 'T', or `ref` is 'G' and `alt` is 'A', otherwise `ref` will be returned from `total_pos_counts`.

    ## Parameters
    ref: str
        Reference genome base.
    alt: str
        Base found in proband.
    include_CpG: bool
        Whether to include CpG DNMs in counts.
    total_pos_counts: Pandas.DataFrame
        DataFrame containing total positional counts (CpG + nonCpG) for `ref`.
    nonCpG_pos_counts: Pandas.DataFrame
        DataFrame containing nonCpG positional counts for `ref`.
    ## Returns
    selected_counts: Numpy.NDArray
        The selected counts.
    '''

    if include_CpG or not ((ref == 'C' and alt == 'T') or (ref == 'G' and alt == 'A')):
        return total_pos_counts[f'{ref}'].to_numpy(dtype=np.float64)
    else:
        return nonCpG_pos_counts[f'{ref}'].to_numpy(dtype=np.float64)


def get_dnm_counts(df: pd.DataFrame, ref: str, alt: str, win_len: int) -> np.ndarray:
    '''
    Return positional DNM counts for the DNM `ref` to `alt`.

    ## Parameters
    df: Pandas.DataFrame
        DataFrame containing DNMs dataset.
    ref: str
        Reference base.
    alt: str
        Base found in proband.
    win_len: int
        Sequence window length.
    ## Returns
    dnm_pos_counts: Numpy.NDArray
        Positional DNM counts.
    '''

    dnm_pos_counts = np.zeros(win_len)

    for dnm in df[['REF','ALT','DIST']][(df.REF == ref) & (df.ALT == alt)].values:
        dist = dnm[-1]
        if dist >= WINDOW_LENGTH//2:
            continue
        
        dnm_pos_counts[dist+win_len//2] += 1

    return dnm_pos_counts


def sliding_window_avg(arr: np.ndarray, sliding_win_len: int, mode: str='same') -> np.ndarray:
    '''
    Return sliding window averages over `arr`.

    ## Parameters
    arr: Numpy.NDArray
        DataFrame containing DNMs dataset.
    sliding_win_len: int
        Sliding window length.
    mode: str, default = 'same'
        Mode used for Numpy's convolve() function. Default mode is 'same', 
        which returns an array with the same size as `arr`.
    ## Returns
    sliding_win_avgs: Numpy.NDArray
        Sliding window averages.
    '''

    # kernel of size sliding_win_len with each element set to 1 / sliding_win_len,
    # which behaves like an averaging filter when used as the kernel during convolution
    kernel_size = np.ones(sliding_win_len) / sliding_win_len

    return np.convolve(
        arr,
        kernel_size,
        mode=mode,
    )

In [141]:
GC_gain_loss_dnm_types = [
	('A','G'),
	('T','G'),
	('A','C'),
	('T','C'),
	('G','A'),
	('G','T'),
	('C','A'),
	('C','T'),
]

metrics_columns = [
	"Number of DNMs",
	"Average of DNMs",
	"Positional count",
	"Rate of mutation",
	"Average rate of mutation",
]

## **7.1** TSS

### **7.1.1** Total (including CpG)

In [300]:
print_positional_metrics(
    df=tss_matches,
    total_pos_counts=tss_pos_counts,
    nonCpG_pos_counts=tss_nonCpG_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    include_CpG=True,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/tss/including_CpG",
    writing_mode=WRITING_MODE,
)

# net GC
print_positional_netGC(
    df=tss_matches,
    pos_counts=tss_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/tss/including_CpG",
    writing_mode=WRITING_MODE,
)

### **7.1.2** non-CpG

In [262]:
print_positional_metrics(
    df=tss_nonCpG,
    total_pos_counts=tss_pos_counts,
    nonCpG_pos_counts=tss_nonCpG_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    include_CpG=False,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/tss/nonCpG",
    writing_mode=WRITING_MODE,
)

# net GC
print_positional_netGC(
    df=tss_nonCpG,
    pos_counts=tss_nonCpG_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/tss/nonCpG",
    writing_mode=WRITING_MODE,
)

## **7.2** Intergenic

### **7.2.1** GC matched

In [286]:
GCmatched_metrics_columns = [
    'Number of DNMs',
    'Positional count',
]

#### Total (including CpG)

In [89]:
for ref, alt in GC_gain_loss_dnm_types:
	raw_tmp = np.zeros(INTERGEN_GCMATCH_WINLEN)

	# dnm count
	for dnm in intergen_GCmatch_matches[['REF','ALT','DIST']][(intergen_GCmatch_matches.REF == ref) & (intergen_GCmatch_matches.ALT == alt)].values:
		dist = dnm[-1]

		raw_tmp[dist+INTERGEN_GCMATCH_WINLEN//2] += 1

	# aggregation
	df_tmp = pd.DataFrame(
		data = [
			raw_tmp,
			intergen_GCmatch_pos_counts[f'{ref}'].loc[-INTERGEN_GCMATCH_WINLEN//2:INTERGEN_GCMATCH_WINLEN//2-1],
		],
	).T.convert_dtypes()
	df_tmp.columns = GCmatched_metrics_columns
	df_tmp.index = range(-INTERGEN_GCMATCH_WINLEN//2, INTERGEN_GCMATCH_WINLEN//2)

	# write out
	with open(f"{POSITIONAL_METRICS_SUBDIR}/intergenic/GC_matched/including_CpG/{ref}{alt}_100b.csv", WRITING_MODE) as fout:
		df_tmp.to_csv(
			fout,
			float_format='%.10f',
			lineterminator='\n',
		)

In [282]:
# net GC
print_positional_netGC(
    df=intergen_GCmatch_matches,
    pos_counts=intergen_GCmatch_pos_counts,
    win_len=INTERGEN_GCMATCH_WINLEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    column_headers=GCmatched_metrics_columns,
    intergenic = True,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/intergenic/GC_matched/including_CpG",
    writing_mode=WRITING_MODE,
)

#### non-CpG

In [91]:
for ref, alt in GC_gain_loss_dnm_types:
	raw_tmp = np.zeros(INTERGEN_GCMATCH_WINLEN)

	# dnm count
	for dnm in intergen_GCmatch_nonCpG[['REF','ALT','DIST']][(intergen_GCmatch_nonCpG.REF == ref) & (intergen_GCmatch_nonCpG.ALT == alt)].values:
		_, _, dist = dnm

		raw_tmp[dist+INTERGEN_GCMATCH_WINLEN//2] += 1

	# aggregation
	df_tmp = pd.DataFrame(
		data = [
			raw_tmp,
			intergen_GCmatch_nonCpG_pos_counts[f'{ref}'].loc[-INTERGEN_GCMATCH_WINLEN//2:INTERGEN_GCMATCH_WINLEN//2-1].to_numpy(dtype=np.float64) if (ref == 'C' and alt == 'T') or (ref == 'G' and alt == 'A') else intergen_GCmatch_pos_counts[f'{ref}'].loc[-INTERGEN_GCMATCH_WINLEN//2:INTERGEN_GCMATCH_WINLEN//2-1].to_numpy(dtype=np.float64),
		],
	).T.convert_dtypes()
	df_tmp.columns = ['Number of DNMs', 'Positional count']
	df_tmp.index = range(-INTERGEN_GCMATCH_WINLEN//2, INTERGEN_GCMATCH_WINLEN//2)

	# write out
	with open(f"{POSITIONAL_METRICS_SUBDIR}/intergenic/GC_matched/non_CpG/{ref}{alt}_100b.csv", WRITING_MODE) as fout:
		df_tmp.to_csv(
			fout,
			float_format='%.10f',
			lineterminator='\n',
		)

In [285]:
# net GC

print_positional_netGC(
    df=intergen_GCmatch_nonCpG,
    pos_counts=intergen_GCmatch_nonCpG_pos_counts,
    win_len=INTERGEN_GCMATCH_WINLEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    column_headers=GCmatched_metrics_columns,
    intergenic = True,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/intergenic/GC_matched/non_CpG",
    writing_mode=WRITING_MODE,
)

### **7.2.2** non-GC matched

#### Total (including CpG)

In [302]:
print_positional_metrics(
    df=intergen_nonGCmatch_matches,
    total_pos_counts=intergen_nonGCmatch_pos_counts,
    nonCpG_pos_counts=intergen_nonGCmatch_nonCpG_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    include_CpG=True,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/intergenic/non_GC_matched/including_CpG",
    writing_mode=WRITING_MODE,
)

# net GC
print_positional_netGC(
    df=intergen_nonGCmatch_matches,
    pos_counts=intergen_nonGCmatch_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/intergenic/non_GC_matched/including_CpG",
    writing_mode=WRITING_MODE,
)

#### non-CpG

In [303]:
print_positional_metrics(
    df=intergen_nonGCmatch_nonCpG,
    total_pos_counts=intergen_nonGCmatch_pos_counts,
    nonCpG_pos_counts=intergen_nonGCmatch_nonCpG_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    include_CpG=False,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/intergenic/non_GC_matched/non_CpG",
    writing_mode=WRITING_MODE,
)

# net GC
print_positional_netGC(
    df=intergen_nonGCmatch_nonCpG,
    pos_counts=intergen_nonGCmatch_nonCpG_pos_counts,
    win_len=WINDOW_LENGTH,
    sliding_win_len=SLIDING_WIN_LEN,
    GC_loss_dnm_types=GC_gain_loss_dnm_types,
    column_headers=metrics_columns,
    parent_dir=f"{POSITIONAL_METRICS_SUBDIR}/intergenic/non_GC_matched/non_CpG",
    writing_mode=WRITING_MODE,
)

________________

# **8.** <u>Gene Level Metrics</u>

In [156]:
gene_identifiers = tss_windows.ENSGID.to_list()

# # write out Ensembl gene IDs (optional)
# gene_identifiers_path = f"{COORDS_DIR}/hg38_genes.csv"
# with open(gene_identifiers_path, WRITING_MODE) as fout:
#     tss_windows.ENSGID.to_csv(
#         fout,
#         lineterminator='\n',
#         index=False,
#         header=False,
#     )

## **8.1** TSS

### **8.1.1** All TSS DNMs (CpG + non-CpG)

In [111]:
tss_gene_DNM_distances_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
								for gene_id
								in gene_identifiers }

for gene_id, dnm_distances in tss_gene_DNM_distances_dict.items():
	for dnm in tss_matches[tss_matches.ENSGID == gene_id].values:
		chrom, pos, ref, alt, tss, eib, dist, first_ex, downstream, intergenic, strand, ensgid, enstid, window = dnm

		dnm_distances['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)

In [112]:
tss_gene_DNM_distances = pd.DataFrame(
	tss_gene_DNM_distances_dict,
).T

tss_gene_DNM_distances

,A>G,T>C,C>G,T>G,C>A,A>T,G>C,G>T,C>T,T>A,A>C,G>A
ENSG00000187634,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000188976,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000187961,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000187583,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000188290,[],[],[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000129862,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000165246,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000012817,[],[],[],[],[],[],[],[],[],[],[],[]
ENSG00000198692,[],[],[],[],[],[],[],[],[],[],[],[]


In [113]:
tss_gene_DNM_distances_repr = tss_gene_DNM_distances.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

tss_gene_DNM_distances_repr = tss_gene_DNM_distances_repr[[
	'A>T',
	'A>G',
	'A>C',
	'T>A',
	'T>G',
	'T>C',
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

# write out results
with open(f"{GENE_LEVEL_METRICS_SUBDIR}/tss/tss_all.csv", WRITING_MODE) as fout:
	tss_gene_DNM_distances_repr.to_csv(
		fout,
		lineterminator='\n',
	)

### **8.1.2** Split CpG and non-CpG TSS DNMs

In [114]:
tss_gene_DNM_distances_CpG_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
									for gene_id
									in gene_identifiers }
tss_gene_DNM_distances_nonCpG_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
										for gene_id
										in gene_identifiers }

for gene_id in gene_identifiers:
	for dnm in tss_matches[tss_matches.ENSGID == gene_id].values:
		chrom, pos, ref, alt, tss, eib, dist, first_ex, downstream, intergenic, strand, ensgid, enstid, window = dnm

		is_CpG = (
			(ref == 'C' and window[WINDOW_LENGTH//2+int(dist)+1+CPG_OFFSET] == 'G') or
			(ref == 'G' and window[WINDOW_LENGTH//2+int(dist)-1+CPG_OFFSET] == 'C')
		)

		if is_CpG:
			tss_gene_DNM_distances_CpG_dict[gene_id]['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)
		else:
			tss_gene_DNM_distances_nonCpG_dict[gene_id]['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)

In [115]:
tss_gene_DNM_distances_CpG = pd.DataFrame(
	tss_gene_DNM_distances_CpG_dict,
).T

tss_gene_DNM_distances_nonCpG = pd.DataFrame(
	tss_gene_DNM_distances_nonCpG_dict,
).T

In [118]:
tss_gene_DNM_distances_CpG_repr = tss_gene_DNM_distances_CpG.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

tss_gene_DNM_distances_CpG_repr = tss_gene_DNM_distances_CpG_repr[[
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/tss/tss_CpG.csv", WRITING_MODE) as fout:
	tss_gene_DNM_distances_CpG_repr.to_csv(
		fout,
		lineterminator='\n',
	)

In [119]:
tss_gene_DNM_distances_nonCpG_repr = tss_gene_DNM_distances_nonCpG.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

tss_gene_DNM_distances_nonCpG_repr = tss_gene_DNM_distances_nonCpG_repr[[
	'A>T',
	'A>G',
	'A>C',
	'T>A',
	'T>G',
	'T>C',
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/tss/tss_nonCpG.csv", WRITING_MODE) as fout:
	tss_gene_DNM_distances_nonCpG_repr.to_csv(
		fout,
		lineterminator='\n',
	)

## **8.2** Intergenic GC matched

### **8.2.1** All intergenic GC matched DNMs (CpG + non-CpG)

In [126]:
intergen_GCmatch_DNM_distances_dict = { f"chr{chrom}:{center-INTERGEN_GCMATCH_WINLEN//2}-{center+INTERGEN_GCMATCH_WINLEN//2}" : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
											for chrom, center, _
											in intergen_GCmatch_windows.values }

for gene_id, dnm_distances in intergen_GCmatch_DNM_distances_dict.items():
	ref_chrom, start, end = re.split('[:-]+',gene_id)
	ref_chrom = re.findall(r'\d+',ref_chrom)[0]

	temp_matches = intergen_GCmatch_matches[
		(intergen_GCmatch_matches.CHROM.astype(str) == ref_chrom) &
		(intergen_GCmatch_matches.CENTER == (int(start) + INTERGEN_GCMATCH_WINLEN//2))
	]

	for dnm in temp_matches.values:
		dnm_chrom, pos, ref, alt, center, dist, window = dnm

		dnm_distances['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)

In [127]:
intergen_GCmatch_DNM_distances = pd.DataFrame(
	intergen_GCmatch_DNM_distances_dict,
).T

intergen_GCmatch_DNM_distances

,A>G,T>C,C>G,T>G,C>A,A>T,G>C,G>T,C>T,T>A,A>C,G>A
chr1:114327737-114327837,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:114328137-114328237,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:114328237-114328337,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:47396304-47396404,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:166420304-166420404,[],[],[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
chr21:8439401-8439501,[],[],[],[],[],[],[],[],[],[],[],[]
chr22:44331786-44331886,[],[],[],[],[],[],[],[],[],[],[],[]
chr22:46071211-46071311,[],[],[],[],[],[],[],[],[],[],[],[]
chr22:16601785-16601885,[],[],[],[],[],[],[],[],[],[],[],[]


In [128]:
intergen_GCmatch_DNM_distances_repr = intergen_GCmatch_DNM_distances.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

intergen_GCmatch_DNM_distances_repr = intergen_GCmatch_DNM_distances_repr[[
	'A>T',
	'A>G',
	'A>C',
	'T>A',
	'T>G',
	'T>C',
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/intergen/GCmatch/intergen_GCmatch_all_100b.csv", WRITING_MODE) as fout:
	intergen_GCmatch_DNM_distances_repr.to_csv(
		fout,
		lineterminator='\n',
	)

### **8.2.2** Split CpG and non-CpG intergenic GC matched DNMs

In [157]:
intergen_GCmatch_DNM_distances_CpG_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
											for gene_id
											in intergen_GCmatch_DNM_distances_dict.keys() }
intergen_GCmatch_DNM_distances_nonCpG_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
												for gene_id
												in intergen_GCmatch_DNM_distances_dict.keys() }


for gene_id in intergen_GCmatch_DNM_distances_dict.keys():
	ref_chrom, start, _ = re.split('[:-]+',gene_id)
	ref_chrom = re.findall(r'\d+',ref_chrom)[0]

	temp_matches = intergen_GCmatch_matches[
		(intergen_GCmatch_matches.CHROM.astype(str) == ref_chrom) &
		(intergen_GCmatch_matches.CENTER == (int(start) + INTERGEN_GCMATCH_WINLEN//2))
	]

	for dnm in temp_matches.values:
		_, _, ref, alt, _, dist, window = dnm

		is_CpG = (
			(ref == 'C' and window[WINDOW_LENGTH//2+int(dist)+1+CPG_OFFSET*0] == 'G') or
			(ref == 'G' and window[WINDOW_LENGTH//2+int(dist)-1+CPG_OFFSET*0] == 'C')
		)

		if is_CpG:
			intergen_GCmatch_DNM_distances_CpG_dict[gene_id]['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)
		else:
			intergen_GCmatch_DNM_distances_nonCpG_dict[gene_id]['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)

In [158]:
intergen_GCmatch_DNM_distances_CpG = pd.DataFrame(
	intergen_GCmatch_DNM_distances_CpG_dict,
).T

intergen_GCmatch_DNM_distances_nonCpG = pd.DataFrame(
	intergen_GCmatch_DNM_distances_nonCpG_dict,
).T

In [159]:
intergen_GCmatch_DNM_distances_CpG_repr = intergen_GCmatch_DNM_distances_CpG.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

intergen_GCmatch_DNM_distances_CpG_repr = intergen_GCmatch_DNM_distances_CpG_repr[[
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/intergen/GCmatch/intergen_GCmatch_CpG_100b.csv", WRITING_MODE) as fout:
	intergen_GCmatch_DNM_distances_CpG_repr.to_csv(
		fout,
		lineterminator='\n',
	)

In [160]:
intergen_GCmatch_DNM_distances_nonCpG_repr = intergen_GCmatch_DNM_distances_nonCpG.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

intergen_GCmatch_DNM_distances_nonCpG_repr = intergen_GCmatch_DNM_distances_nonCpG_repr[[
	'A>T',
	'A>G',
	'A>C',
	'T>A',
	'T>G',
	'T>C',
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/intergen/GCmatch/intergen_GCmatch_nonCpG_100b.csv", WRITING_MODE) as fout:
	intergen_GCmatch_DNM_distances_nonCpG_repr.to_csv(
		fout,
		lineterminator='\n',
	)

## **8.3** Intergenic non-GC matched

### **8.3.1** All intergenic non-GC matched DNMs (CpG + non-CpG)

In [146]:
intergen_nonGCmatch_DNM_distances_dict = { f"chr{chrom}:{center-WINDOW_LENGTH//2}-{center+WINDOW_LENGTH//2}" : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
											for chrom, center, window
											in intergen_nonGCmatch_windows.values }

for gene_id, dnm_distances in intergen_nonGCmatch_DNM_distances_dict.items():
	ref_chrom, start, end = re.split('[:-]+',gene_id)
	ref_chrom = re.findall(r'\d+',ref_chrom)[0]

	temp_matches = intergen_nonGCmatch_matches[
		(intergen_nonGCmatch_matches.CHROM.astype(str) == ref_chrom) &
		(intergen_nonGCmatch_matches.CENTER == (int(start) + WINDOW_LENGTH//2))
	]

	for dnm in temp_matches.values:
		dnm_chrom, pos, ref, alt, center, dist, window = dnm

		dnm_distances['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)

In [147]:
intergen_nonGCmatch_DNM_distances = pd.DataFrame(
	intergen_nonGCmatch_DNM_distances_dict,
).T

intergen_nonGCmatch_DNM_distances

,A>G,T>C,C>G,T>G,C>A,A>T,G>C,G>T,C>T,T>A,A>C,G>A
chr1:114326537-114331537,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:47396104-47401104,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:166419304-166424304,[],[],[],[],[],[],[],[],[],[],[],[]
chr1:69212604-69217604,[],[],[],[],[],[3671],[],[],[],[],[],[]
chr1:29409919-29414919,[],[],[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
chr22:11484544-11489544,[],[],[],[],[],[],[],[],[],[],[],[]
chr22:27191025-27196025,[],[],[],[],[],[],[],[],[],[],[],[]
chr22:12061784-12066784,[],[],[],[],[],[],[],[],[],[],[],[]
chr22:29301107-29306107,[],[],[],[],[],[],[],[],[],[],[],[3895]


In [148]:
intergen_nonGCmatch_DNM_distances_repr = intergen_nonGCmatch_DNM_distances.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

intergen_nonGCmatch_DNM_distances_repr = intergen_nonGCmatch_DNM_distances_repr[[
	'A>T',
	'A>G',
	'A>C',
	'T>A',
	'T>G',
	'T>C',
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/intergen/nonGCmatch/intergen_nonGCmatch_all.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_DNM_distances_repr.to_csv(
		fout,
		lineterminator='\n',
	)

### **8.3.2** Split CpG and non-CpG intergenic non-GC matched DNMs

In [149]:
intergen_nonGCmatch_DNM_distances_CpG_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
												for gene_id
												in intergen_nonGCmatch_DNM_distances_dict.keys() }
intergen_nonGCmatch_DNM_distances_nonCpG_dict = { gene_id : dict(zip(DNM_TYPES,[list() for i in range(len(DNM_TYPES))]))
													for gene_id
													in intergen_nonGCmatch_DNM_distances_dict.keys() }


for gene_id in intergen_nonGCmatch_DNM_distances_dict.keys():
	ref_chrom, start, _ = re.split('[:-]+',gene_id)
	ref_chrom = re.findall(r'\d+',ref_chrom)[0]

	temp_matches = intergen_nonGCmatch_matches[
		(intergen_nonGCmatch_matches.CHROM.astype(str) == ref_chrom) &
		(intergen_nonGCmatch_matches.CENTER == (int(start) + WINDOW_LENGTH//2))
	]

	for dnm in temp_matches.values:
		_, _, ref, alt, _, dist, window = dnm

		is_CpG = (
			(ref == 'C' and window[WINDOW_LENGTH//2+int(dist)+1+CPG_OFFSET*0] == 'G') or
			(ref == 'G' and window[WINDOW_LENGTH//2+int(dist)-1+CPG_OFFSET*0] == 'C')
		)

		if is_CpG:
			intergen_nonGCmatch_DNM_distances_CpG_dict[gene_id]['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)
		else:
			intergen_nonGCmatch_DNM_distances_nonCpG_dict[gene_id]['>'.join([ref, alt])].append(dist+WINDOW_LENGTH//2)

In [150]:
intergen_nonGCmatch_DNM_distances_CpG = pd.DataFrame(
	intergen_nonGCmatch_DNM_distances_CpG_dict,
).T

intergen_nonGCmatch_DNM_distances_nonCpG = pd.DataFrame(
	intergen_nonGCmatch_DNM_distances_nonCpG_dict,
).T

In [153]:
intergen_nonGCmatch_DNM_distances_CpG_repr = intergen_nonGCmatch_DNM_distances_CpG.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

intergen_nonGCmatch_DNM_distances_CpG_repr = intergen_nonGCmatch_DNM_distances_CpG_repr[[
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/intergen/nonGCmatch/intergen_nonGCmatch_CpG.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_DNM_distances_CpG_repr.to_csv(
		fout,
		lineterminator='\n',
	)

In [154]:
intergen_nonGCmatch_DNM_distances_nonCpG_repr = intergen_nonGCmatch_DNM_distances_nonCpG.astype(str).apply(lambda x: x.apply(lambda y: y.strip('[]')))

intergen_nonGCmatch_DNM_distances_nonCpG_repr = intergen_nonGCmatch_DNM_distances_nonCpG_repr[[
	'A>T',
	'A>G',
	'A>C',
	'T>A',
	'T>G',
	'T>C',
	'G>A',
	'G>T',
	'G>C',
	'C>A',
	'C>T',
	'C>G',
]]

with open(f"{GENE_LEVEL_METRICS_SUBDIR}/intergen/nonGCmatch/intergen_nonGCmatch_nonCpG.csv", WRITING_MODE) as fout:
	intergen_nonGCmatch_DNM_distances_nonCpG_repr.to_csv(
		fout,
		lineterminator='\n',
	)